In [1]:
import os
import json
import ray
import numpy as np
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.networks import Network
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, VehicleParams, SumoCarFollowingParams, InFlows, SumoLaneChangeParams
from flow.controllers import RLController, IDMController, SimLaneChangeController
from gym.spaces.box import Box
from gym.spaces import Tuple

from constructionRouter import ConstructionRouter
from env_construction_padding import myEnv

# time horizon of a single rollout
HORIZON = 5000
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 2

vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             num_vehicles=10
             )
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode=25
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode=1621),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE43"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE43": ["gneE43","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
               "gneE8": ["gneE8","gneE9","gneE37","gneE38","gneE39","gneE4.264.110","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE43",
           vehs_per_hour=10000,
            depart_lane="random",
            depart_speed="random",
            color="white")

file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/construction/constructionV4.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.5,
        render=False,
        restart_instance=True
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=1000,
        clip_actions=False,
        additional_params={
            "target_velocity": 50,
            "sort_vehicles": False,
            "max_accel": 2,
            "max_decel": 2,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [16, 16, 16, 16]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config


/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 5,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})

2020-03-25 09:39:07,853	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-25 09:39:07,860	INFO resource_spec.py:212 -- Starting Ray with 4.74 GiB memory available for workers and up to 2.38 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-25 09:39:08,395	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_013aecf2,RUNNING,


(pid=599) E0325 09:39:10.381715200     599 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1585143550.381689500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=599) E0325 09:39:10.382018700     599 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0325 09:39:08.625950200     578 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1585143548.625917500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0325 09:39:08.626523800     578 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=rayle

(pid=601) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=601)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=601) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=601)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=601) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=601)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid

(pid=601) Warning: Vehicle 'flow_00.214' performs emergency braking with decel=-12.02 wished=4.50 severity=1.67, time=234.50.
(pid=601) Warning: Vehicle 'flow_00.200' performs emergency braking with decel=-11.62 wished=4.50 severity=1.58, time=235.00.
(pid=601) Warning: Vehicle 'flow_00.220' performs emergency braking with decel=-12.08 wished=4.50 severity=1.69, time=235.50.
(pid=601) Warning: Vehicle 'flow_00.222' performs emergency braking with decel=-13.38 wished=4.50 severity=1.97, time=236.00.
(pid=601) Warning: Vehicle 'flow_00.218' performs emergency braking with decel=-10.64 wished=4.50 severity=1.37, time=236.50.
(pid=600) Warning: Vehicle 'rl_2' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=255.00.
(pid=601) Warning: Vehicle 'flow_00.168' performs emergency braking with decel=-13.61 wished=4.50 severity=2.03, time=262.00.
(pid=601) Warning: Vehicle 'flow_00.178' performs emergency braking with decel=-9.01 wished=4.50 severity=1.00, time=262.50.
(

...

(pid=601) Warning: Vehicle 'flow_00.894' performs emergency braking with decel=-15.87 wished=4.50 severity=2.53, time=880.00.
(pid=601) Warning: Vehicle 'flow_00.889' performs emergency braking with decel=-13.02 wished=4.50 severity=1.89, time=880.50.
(pid=601) Warning: Vehicle 'flow_00.916' performs emergency braking with decel=-10.74 wished=4.50 severity=1.39, time=941.50.
(pid=601) Warning: Teleporting vehicle 'flow_00.461'; collision with vehicle 'rl_8', lane=':gneJ30_2_1', gap=-1.00, time=1089.00 stage=move.
(pid=601) VALUE ERROR VEL: OUTSIDE RANGE -1073741824.0
(pid=601) Warning: Vehicle 'rl_0' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=108.50.
(pid=601) Warning: Vehicle 'rl_4' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=114.50.
(pid=601) Warning: Vehicle 'flow_00.9' performs emergency braking with decel=-28.98 wished=4.50 severity=5.44, time=152.50.
(pid=601) Warning: Vehicle 'flow_00.7' performs emergency braking 

(pid=600) Warning: Vehicle 'rl_4' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=264.00.
(pid=600) Warning: Vehicle 'rl_1' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=300.50.
(pid=600) Warning: Vehicle 'flow_00.269' performs emergency braking with decel=-10.31 wished=4.50 severity=1.29, time=303.00.
(pid=600) Warning: Vehicle 'flow_00.280' performs emergency braking with decel=-9.95 wished=4.50 severity=1.21, time=303.50.
(pid=600) Warning: Vehicle 'flow_00.296' performs emergency braking with decel=-12.54 wished=4.50 severity=1.79, time=304.50.
(pid=600) Warning: Vehicle 'flow_00.308' performs emergency braking with decel=-14.01 wished=4.50 severity=2.11, time=305.00.
(pid=600) Warning: Vehicle 'flow_00.293' performs emergency braking with decel=-15.38 wished=4.50 severity=2.42, time=305.50.
(pid=600) Warning: Vehicle 'flow_00.311' performs emergency braking with decel=-15.96 wished=4.50 severity=2.55, time=306.00.
(pid=600)

(pid=600) Warning: Vehicle 'flow_00.255' performs emergency braking with decel=-14.26 wished=4.50 severity=2.17, time=279.00.
(pid=600) Warning: Vehicle 'flow_00.251' performs emergency braking with decel=-14.76 wished=4.50 severity=2.28, time=279.50.
(pid=600) Warning: Vehicle 'flow_00.248' performs emergency braking with decel=-15.06 wished=4.50 severity=2.35, time=280.00.
(pid=600) Warning: Vehicle 'flow_00.244' performs emergency braking with decel=-12.91 wished=4.50 severity=1.87, time=280.50.
(pid=600) Warning: Vehicle 'flow_00.262' performs emergency braking with decel=-15.34 wished=4.50 severity=2.41, time=280.50.
(pid=600) Warning: Vehicle 'flow_00.247' performs emergency braking with decel=-15.95 wished=4.50 severity=2.54, time=281.00.
(pid=600) Warning: Vehicle 'flow_00.268' performs emergency braking with decel=-15.27 wished=4.50 severity=2.39, time=281.00.
(pid=600) Warning: Vehicle 'flow_00.259' performs emergency braking with decel=-17.09 wished=4.50 severity=2.80, time=

(pid=601) Warning: Teleporting vehicle 'rl_4'; collision with vehicle 'rl_3', lane=':gneJ31_0_0', gap=-1.00, time=144.50 stage=move.
(pid=601) Warning: Vehicle 'rl_4' ends teleporting on edge 'gneE37', time 144.50.
(pid=601) Warning: Vehicle 'flow_00.37' performs emergency braking with decel=-9.77 wished=4.50 severity=1.17, time=147.50.
(pid=601) Warning: Teleporting vehicle 'flow_00.6'; collision with vehicle 'rl_2', lane=':gneJ31_3_0', gap=-1.00, time=151.00 stage=move.
(pid=601) Warning: Vehicle 'flow_00.6' ends teleporting on edge 'gneE9.252', time 151.00.
(pid=601) Warning: Vehicle 'flow_00.0' performs emergency braking with decel=-21.82 wished=4.50 severity=3.85, time=151.50.
(pid=601) Warning: Vehicle 'flow_00.10' performs emergency braking with decel=-9.44 wished=4.50 severity=1.10, time=152.00.
(pid=601) Warning: Vehicle 'rl_9' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=214.00.
(pid=601) Warning: Vehicle 'rl_6' performs emergency braking with d

(pid=600) Warning: Vehicle 'flow_00.582' performs emergency braking with decel=-10.99 wished=4.50 severity=1.44, time=581.50.
(pid=600) Warning: Vehicle 'flow_00.626' performs emergency braking with decel=-21.69 wished=4.50 severity=3.82, time=614.00.
(pid=600) Warning: Vehicle 'flow_00.632' performs emergency braking with decel=-9.50 wished=4.50 severity=1.11, time=614.50.
(pid=601) Warning: Vehicle 'flow_00.426' performs emergency braking with decel=-18.53 wished=4.50 severity=3.12, time=470.50.
(pid=601) Warning: Vehicle 'flow_00.567' performs emergency braking with decel=-12.92 wished=4.50 severity=1.87, time=562.50.
